In [7]:
import pandas
import numpy
from numpy import std, correlate, sqrt
from scipy.stats import pearsonr
import statsmodels.api as sm
from statsmodels.stats.mediation import Mediation

### Preparing data

In [2]:
filepath = 'D:/DEA/Data/Original_data.txt'
data = pandas.read_csv(filepath,sep='\t')
data.rename(columns={'origi':'Id'},inplace=True)
print(data.shape,'from 2019_05_25_Results.xlxs')
print(data.Year.unique(),'Years')
print(data.Id.nunique(),'Number of firms')
filepath = 'D:/DEA/Data/Database_secondreviewJIBS.dta' #
data1 = pandas.read_stata(filepath)
data1.rename(columns={'id':'Id','year':'Year','country':'Country'},inplace=True)
data1 = data1[['Id','Year','Country']]
data1['Year'] = pandas.to_numeric(data1.Year.astype(str).str[0:4])
print(data1.shape,'to get country')
data = pandas.merge(left=data,right=data1,how='left',on=['Year','Id'])
print(data.Country.unique())
filepath = 'D:/DEA/Data/2019_05_25_Results_updated.txt'
data2 = pandas.read_csv(filepath,sep='\t')
print(data2.shape,'correction by Elio of market commonality')
data2.rename(columns={'origi':'Id'},inplace=True)
data2 = data2[['Id','Year','Market_commonality']]
data.drop('Market_commonality',axis=1,inplace = True)
data = pandas.merge(left=data,right=data2,how='inner',on=['Year','Id'])
print(data.Non_RD_alliances.sum(),'Total number of Non-RD alliances')
print(data.RD_alliances.sum(),'Total number of RD alliances')

(1232, 33) from 2019_05_25_Results.xlxs
[1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
 2005 2006 2007 2008 2009 2010 2011 2012] Years
56 Number of firms
(1539, 3) to get country
[nan 'USA' 'Switzerland' 'Netherlands' 'Germany' 'South Africa' 'Japan'
 'United Kingdom' 'France' 'Australia' 'Ireland' 'Spain' 'Denmark'
 'Sweden']
(1232, 34) correction by Elio of market commonality
4089.0 Total number of Non-RD alliances
7676.0 Total number of RD alliances


### Creating variables

In [3]:
df = data.copy()
df = df.sort_values(by=['Id','Year'],axis=0,ascending=True)
# ASPIRATIONS
df['SA']=df.groupby(['Year']).ROE.transform('mean')
df['distance'] = df['ROE'] - df['SA']
df.loc[df.distance<0,'BSA']=df['distance']
df.loc[(df.distance>0)|(df.distance==0),'BSA']=0
df.loc[(df.distance>0)|(df.distance==0),'ASA']=df['distance']
df.loc[df.distance<0,'ASA']=0
# DURATION
df['DURATION'] = 0
mylist = []
for firm in range(df.Id.nunique()):
  mylist.append(df.loc[df.Id==firm+1,['Id','Year','BSA','DURATION']].values.tolist())
m = [0]
for firm in range(df.Id.nunique()):
  for i in mylist[firm]: 
    if i[2]<0:
      m.append([i[0],i[1],m[-1][2]+1])
    else:
      m.append([i[0],i[1],i[3]])
del m[0]
print(len(m))
mydf = pandas.DataFrame(m,columns=['Id','Year','DURATION'])
df = df.drop('DURATION',axis=1)
df = pandas.merge(left=df,right=mydf,how='inner',on=['Year','Id'])
print(df.loc[(df.Id==38)|(df.Id==39),['Id','Year','BSA','DURATION']])
df = df.sort_values(by=['Id','Year'], axis=0, ascending=True)
#df['BSA'] = df.groupby(['Id'])['BSA'].shift(1)
#df['ASA'] = df.groupby(['Id'])['ASA'].shift(1)
#df['DURATION'] = df.groupby(['Id'])['DURATION'].shift(1)
# ATTENTION
filepath = 'D:/DEA/Data/Results_def_01_for_Elio.txt'
data5 = pandas.read_csv(filepath,sep='\t')
data5 = data5.iloc[1:1008,[0,1,25,26,27]]
data5.columns = ['Year','Id','PRODUCT','GEOGRAPHY','GROWTH']
data5['Year'] = pandas.to_numeric(data5.Year.astype(str).str[0:4])
print(data5)
df = pandas.merge(left=df,right=data5,how='inner',on=['Year','Id'])
df['ATTENTION'] = df[['PRODUCT','GEOGRAPHY','GROWTH']].std(axis=1)
# AMBIDEXTERITY
print(df.Non_RD_alliances.corr(df.RD_alliances),'correlation between RD alliances and non-RD alliances')
df['RD'] = df.RD_alliances
# SIZE
df['Size'] = numpy.log(df.Employees) 
df = df.sort_values(by=['Id','Year'], axis=0, ascending=True)
df[17:30]

1232
     Id  Year       BSA  DURATION
814  38  1991  0.000000       0.0
815  38  1992  0.000000       0.0
816  38  1993  0.000000       0.0
817  38  1994  0.000000       0.0
818  38  1995 -0.014391       1.0
819  38  1996 -0.098000       2.0
820  38  1997  0.000000       0.0
821  38  1998 -0.209075       1.0
822  38  1999 -0.059585       2.0
823  38  2000 -0.124833       3.0
824  38  2001 -0.008870       4.0
825  38  2002  0.000000       0.0
826  38  2003  0.000000       0.0
827  38  2004 -0.045434       1.0
828  38  2005 -0.000642       2.0
829  38  2006 -0.000154       3.0
830  38  2007 -0.634098       4.0
831  38  2008 -0.259360       5.0
832  38  2009 -0.163826       6.0
833  38  2010 -0.089444       7.0
834  38  2011 -0.104333       8.0
835  38  2012 -0.025738       9.0
836  39  1991       NaN       0.0
837  39  1992       NaN       0.0
838  39  1993       NaN       0.0
839  39  1994       NaN       0.0
840  39  1995       NaN       0.0
841  39  1996 -0.111000       1.0
842  39  

,Id,Year,Age,Employees,Total_assets,Net_sales,Human_resources_1,Human_resources_2,Physical_resources_1,Physical_resources_2,...,distance,BSA,ASA,DURATION,PRODUCT,GEOGRAPHY,GROWTH,ATTENTION,RD,Size
17,2,1995,107.0,50241.0,9412.580,10012.194,0.199,5.018,4249.538,0.424,...,0.203609,0.000000,0.203609,0.0,0.0,0.339996,0.660004,0.330052,9.0,10.824587
18,2,1996,108.0,52817.0,11125.600,11013.460,0.209,4.796,4461.543,0.405,...,0.196000,0.000000,0.196000,0.0,0.0,0.407071,0.592929,0.303264,12.0,10.874588
19,2,1997,109.0,54487.0,12061.068,11883.462,0.218,4.585,4569.691,0.385,...,0.289423,0.000000,0.289423,0.0,0.0,0.440513,0.559487,0.294741,23.0,10.905717
20,2,1998,110.0,56236.0,13216.213,12477.845,0.222,4.507,4738.843,0.380,...,0.089925,0.000000,0.089925,0.0,0.0,0.430937,0.569063,0.296821,11.0,10.937312
21,2,1999,111.0,57100.0,14471.044,13177.625,0.231,4.333,4770.059,0.362,...,0.141415,0.000000,0.141415,0.0,0.0,0.435399,0.564601,0.295815,13.0,10.952559
22,2,2000,112.0,60571.0,15283.254,13745.916,0.227,4.406,4816.911,0.350,...,0.167167,0.000000,0.167167,0.0,0.0,0.448690,0.551310,0.293200,25.0,11.011572
23,2,2001,113.0,71426.0,23296.423,16285.246,0.228,4.386,5551.547,0.341,...,-0.023870,-0.023870,0.000000,1.0,0.0,0.299710,0.700290,0.351354,21.0,11.176417
24,2,2002,114.0,71819.0,24259.102,17684.663,0.246,4.061,5828.122,0.330,...,0.121444,0.000000,0.121444,0.0,0.0,0.534176,0.465824,0.290691,12.0,11.181904
25,2,2003,115.0,72181.0,26715.342,19680.561,0.273,3.668,6281.806,0.319,...,0.058906,0.000000,0.058906,0.0,0.0,0.504841,0.495159,0.288716,9.0,11.186932
26,2,2004,116.0,60600.0,28767.494,19680.016,0.325,3.079,6007.874,0.305,...,0.070566,0.000000,0.070566,0.0,0.0,0.479917,0.520083,0.289373,21.0,11.012050


### Correlation

In [9]:
mydata = df[['RD','BSA','DURATION','GROWTH','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
print(len(mydata),'Number of observations')
ds_mean = mydata.agg(['mean','std']).T
corr = mydata.corr()
corr = pandas.concat([ds_mean,corr], axis = 1)
corr = corr.applymap('{:.2f}'.format)
corr['statistics'] = ' '
corr.set_index('statistics',append=True,inplace=True)
print(corr)

def pearsonr_pval(x,y):
  return pearsonr(x,y)[1].round(3)
pval = mydata.corr(method=pearsonr_pval)
pval = pval.applymap('{:.3f}'.format)
pval = pval.applymap(lambda x: '(' + x + ')')
pval['statistics'] = 'p-value'
pval.set_index('statistics',append=True,inplace=True)

corr = pandas.concat([corr,pval],axis=0).sort_index(kind='merge').reindex(['RD','BSA','DURATION','GROWTH','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year'],axis=0,level=0)
filepath = 'D:/DEA/DEA-2/Correlation-0.txt'
corr.to_csv(filepath,sep = '\t',header=False,index=False)
corr

1007 Number of observations
                                   mean    std     RD    BSA DURATION GROWTH  \
                    statistics                                                 
RD                                 8.04   8.55   1.00   0.14    -0.20   0.06   
BSA                               -0.10   0.21   0.14   1.00    -0.19   0.05   
DURATION                           3.67   6.52  -0.20  -0.19     1.00  -0.02   
GROWTH                             0.15   0.16   0.06   0.05    -0.02   1.00   
ASA                                0.10   0.33   0.06   0.14    -0.18   0.11   
Absorbed_slack                     1.97  37.88  -0.04  -0.06    -0.01  -0.19   
Unabsorbed_slack                   1.39   1.99  -0.18  -0.16     0.06  -0.09   
Absorptive_capacity                0.23   0.72  -0.07  -0.46     0.06  -0.12   
Market_commonality                 0.41   0.14   0.01  -0.08     0.03   0.12   
Size                               9.37   1.76   0.41   0.30    -0.16   0.18   
Year        

mean    std       RD      BSA DURATION  \
                    statistics                                              
RD                                 8.04   8.55     1.00     0.14    -0.20   
                    p-value         NaN    NaN  (1.000)  (0.000)  (0.000)   
BSA                               -0.10   0.21     0.14     1.00    -0.19   
                    p-value         NaN    NaN  (0.000)  (1.000)  (0.000)   
DURATION                           3.67   6.52    -0.20    -0.19     1.00   
                    p-value         NaN    NaN  (0.000)  (0.000)  (1.000)   
GROWTH                             0.15   0.16     0.06     0.05    -0.02   
                    p-value         NaN    NaN  (0.175)  (0.220)  (0.719)   
ASA                                0.10   0.33     0.06     0.14    -0.18   
                    p-value         NaN    NaN  (0.102)  (0.000)  (0.000)   
Absorbed_slack                     1.97  37.88    -0.04    -0.06    -0.01   
                    p-value         NaN    NaN  (0.322)  (0.094)  (0.760)   
Unabsorbed_slack                   1.39   1.99    -0.18    -0.16     0.06   
                    p-value         NaN    NaN  (0.000)  (0.000)  (0.132)   
Absorptive_capacity                0.23   0.72    -0.07    -0.46     0.06   
                    p-value         NaN    NaN  (0.046)  (0.000)  (0.055)   
Market_commonality                 0.41   0.14     0.01    -0.08     0.03   
                    p-value         NaN    NaN  (0.771)  (0.014)  (0.372)   
Size                               9.37   1.76     0.41     0.30    -0.16   
                    p-value         NaN    NaN  (0.000)  (0.000)  (0.000)   
Year                            2003.51   5.19    -0.14     0.04     0.09   
                    p-value         NaN    NaN  (0.000)  (0.211)  (0.003)   

                                 GROWTH      ASA Absorbed_slack  \
                    statistics                                    
RD                                 0.06     0.06          -0.04   
                    p-value     (0.175)  (0.102)        (0.322)   
BSA                                0.05     0.14          -0.06   
                    p-value     (0.220)  (0.000)        (0.094)   
DURATION                          -0.02    -0.18          -0.01   
                    p-value     (0.719)  (0.000)        (0.760)   
GROWTH                             1.00     0.11          -0.19   
                    p-value     (1.000)  (0.010)        (0.000)   
ASA                                0.11     1.00          -0.01   
                    p-value     (0.010)  (1.000)        (0.722)   
Absorbed_slack                    -0.19    -0.01           1.00   
                    p-value     (0.000)  (0.722)        (1.000)   
Unabsorbed_slack                  -0.09     0.10           0.01   
                    p-value     (0.070)  (0.007)        (0.771)   
Absorptive_capacity               -0.12     0.13           0.00   
                    p-value     (0.004)  (0.000)        (0.952)   
Market_commonality                 0.12    -0.02          -0.02   
                    p-value     (0.005)  (0.560)        (0.662)   
Size                               0.18     0.04          -0.10   
                    p-value     (0.000)  (0.296)        (0.014)   
Year                              -0.06    -0.03           0.07   
                    p-value     (0.175)  (0.416)        (0.069)   

                               Unabsorbed_slack Absorptive_capacity  \
                    statistics                                        
RD                                        -0.18               -0.07   
                    p-value             (0.000)             (0.046)   
BSA                                       -0.16               -0.46   
                    p-value             (0.000)             (0.000)   
DURATION                                   0.06                0.06   
                    p-value             (0.132)             (0.055)   
GROWTH   

## Regression - Mediation

In [11]:
mydata = df[['Id','RD','BSA','DURATION','GROWTH','ASA','Absorbed_slack','Unabsorbed_slack','Absorptive_capacity','Market_commonality','Size','Year']]
mydata = mydata.dropna()
mydata = (mydata - mydata.min()) / (mydata.max() - mydata.min()) 
mydata = mydata.sort_values(by=['Id','Year'], axis=0, ascending=True)
mediator_model = sm.OLS.from_formula('GROWTH~BSA', data=mydata)
outcome_model = sm.OLS.from_formula('RD~BSA+DURATION+GROWTH+ASA+Absorbed_slack+Unabsorbed_slack+Absorptive_capacity+Market_commonality+Size+C(Id)+C(Year)',data=mydata)   
mediator = mediator_model.fit(cov_type='HC1')
print(mediator.summary())
outcome = outcome_model.fit(cov_type='hac-panel', cov_kwds={'maxlags':1, 'groups':mydata['Id']})
print(outcome.summary())
effects = Mediation(outcome_model,mediator_model,"BSA","GROWTH").fit(n_rep=500)
effects.summary()

                            OLS Regression Results                            
Dep. Variable:                 GROWTH   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.869
Date:                Thu, 11 Mar 2021   Prob (F-statistic):             0.0911
Time:                        11:04:29   Log-Likelihood:                 9.2531
No. Observations:                 374   AIC:                            -14.51
Df Residuals:                     372   BIC:                            -6.658
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0334      0.109      0.307      0.7

,Estimate,Lower CI bound,Upper CI bound,P-value
ACME (control),0.013719,-0.022411,0.062770,0.468
ACME (treated),0.013719,-0.022411,0.062770,0.468
ADE (control),0.025921,-0.137469,0.179849,0.724
ADE (treated),0.025921,-0.137469,0.179849,0.724
Total effect,0.039641,-0.122476,0.193913,0.624
Prop. mediated (control),0.064899,-2.500307,2.280634,0.740
Prop. mediated (treated),0.064899,-2.500307,2.280634,0.740
ACME (average),0.013719,-0.022411,0.062770,0.468
ADE (average),0.025921,-0.137469,0.179849,0.724
Prop. mediated (average),0.064899,-2.500307,2.280634,0.740
